In [1]:
! pip install whisper
! pip install -U openai-whisper
!choco install ffmpeg
!pip install setuptools-rust

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=fa4cf56188ce87244ce286e85c57f07498a0143d49c3599e53d4ac5f0deb740b
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 32.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=c5cf77197d0d633a3f1ead0cbc824aff16b89263016fc6bda102f633b90c488c
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built o

In [ ]:
torch.cuda.empty_cache()

In [2]:
!nvidia-smi

Tue Jan 28 15:43:47 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import gc
import torch

# For TensorFlow/Keras:
from tensorflow.keras import backend as K
K.clear_session()

# For PyTorch:
torch.cuda.empty_cache()

# Force garbage collection:
gc.collect()


0

In [6]:
import torch
import whisper
import gc
from tqdm import tqdm
import time
from datetime import datetime, timedelta

class OptimizedTranscriber:
    def __init__(self, model_name="large"):
        self.start_time = time.time()
        print(f"Initializing transcriber at {datetime.now().strftime('%H:%M:%S')}")

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Enable TensorFloat-32 for better performance on Ampere GPUs
        if torch.cuda.is_available():
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

        # Load model with optimizations
        self.model = whisper.load_model(model_name)
        self.model.to(self.device)

        if torch.cuda.is_available():
            # Convert model to half precision
            self.model = self.model.half()

        self.model_load_time = time.time() - self.start_time
        print(f"Model loading took: {timedelta(seconds=self.model_load_time)}")

    def transcribe(self, audio_path):
        start_time = time.time()
        print(f"\nStarting transcription of {audio_path} at {datetime.now().strftime('%H:%M:%S')}")

        try:
            # Enable cuda graphs for repeated operations
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            # Use modern autocast syntax
            with torch.amp.autocast('cuda' if torch.cuda.is_available() else 'cpu'), \
                 torch.no_grad():

                result = self.model.transcribe(
                    audio_path,
                    fp16=torch.cuda.is_available(),
                    verbose=True
                )

            # Force CUDA synchronization
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            process_time = time.time() - start_time
            print(f"Transcription completed in: {timedelta(seconds=process_time)}")

            return result, process_time

        finally:
            # Clean up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()

    def process_multiple(self, audio_files):
        """Process multiple audio files sequentially with progress bar"""
        results = []
        total_time = 0
        start_batch = time.time()

        for audio_file in tqdm(audio_files):
            result, process_time = self.transcribe(audio_file)
            results.append(result)
            total_time += process_time

        batch_time = time.time() - start_batch
        print(f"\nBatch Processing Summary:")
        print(f"Total files processed: {len(audio_files)}")
        print(f"Total processing time: {timedelta(seconds=batch_time)}")
        print(f"Average time per file: {timedelta(seconds=batch_time/len(audio_files))}")

        return results

def format_time(seconds):
    return str(timedelta(seconds=round(seconds)))

# Usage
# def main():
total_start = time.time()

# Initialize transcriber
transcriber = OptimizedTranscriber(model_name="large")

# Single file transcription
result, process_time = transcriber.transcribe("/content/full.mp3")

print("\n--- Full Transcription ---")
print(result["text"])

print("\n--- Transcription Details ---")
print(f"Language Detected: {result['language']}")
print(f"Approximate Duration: {len(result['segments'])} segments")
print(f"Processing Time: {format_time(process_time)}")

total_time = time.time() - total_start
print(f"\nTotal execution time (including model loading): {format_time(total_time)}")

    # For processing multiple files:
    # audio_files = ["/content/1.mp3", "/content/2.mp3", ...]
    # results = transcriber.process_multiple(audio_files)

# if __name__ == "__main__":
#     main()

Initializing transcriber at 15:47:48
Using device: cuda


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 21752 has 14.73 GiB memory in use. Of the allocated memory 13.79 GiB is allocated by PyTorch, and 834.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
!nvidia-smi

Tue Jan 28 15:49:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              30W /  70W |  15085MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
result, process_time = transcriber.transcribe("/content/full.mp3")


Starting transcription of /content/full.mp3 at 15:48:50
Detecting language using up to the first 30 seconds. Use `--language` to specify the language


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 21752 has 14.73 GiB memory in use. Of the allocated memory 13.79 GiB is allocated by PyTorch, and 826.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import threading
import time

def get_gpu_memory():
    """Get the current GPU memory usage."""
    result = os.popen("nvidia-smi --query-gpu=memory.used --format=csv,nounits,noheader").read()
    return int(result.strip().split('\n')[0])  # Return GPU memory in MB

def monitor_gpu_memory(stop_event):
    """Monitor GPU memory usage in real-time."""
    while not stop_event.is_set():
        gpu_memory = get_gpu_memory()
        print(f"Real-time GPU memory usage: {gpu_memory} MB")
        time.sleep(1)  # Log every 1 second

# Create a stop event for the monitoring thread
stop_event = threading.Event()

# Start the monitoring thread
monitor_thread = threading.Thread(target=monitor_gpu_memory, args=(stop_event,))
monitor_thread.start()

try:
    # Run the transcription process
    print("Starting transcription...")
    result, process_time = transcriber.transcribe("/content/full.mp3")
    print("Transcription completed!")
    print(result["text"])
finally:
    # Stop the monitoring thread
    stop_event.set()
    monitor_thread.join()

# Get GPU memory after running the transcription
gpu_memory_after = get_gpu_memory()
print(f"Final GPU memory usage: {gpu_memory_after} MB")


Starting transcription...

Starting transcription of /content/full.mp3 at 11:39:50
Real-time GPU memory usage: 3943 MB
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
Real-time GPU memory usage: 4017 MB
Real-time GPU memory usage: 4023 MB
[00:00.640 --> 00:04.400]  Albert Einstein grew up in a middle class family.
[00:04.400 --> 00:11.920]  When he was born in his family was scared that something was wrong with him and he had
[00:11.920 --> 00:19.120]  very large in miss shape and head.
[00:19.120 --> 00:27.000]  But fortunately within the first few weeks the shape of his head became normal.
Real-time GPU memory usage: 4027 MB
Real-time GPU memory usage: 4033 MB
Real-time GPU memory usage: 4041 MB
[00:27.000 --> 00:32.620]  But their worries didn't stop there when he was very young.
[00:32.620 --> 00:40.760]  His parents thought he might be intellectually disabled because he was very slow to learn
[00:40.760 --> 0

In [ ]:
!nvidia-smi

Tue Jan 28 11:39:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              32W /  70W |   3943MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## initiate flask with Wispher

In [1]:
! pip install whisper
! pip install -U openai-whisper
!choco install ffmpeg
!pip install setuptools-rust

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=3c3f8de68d7d249a9fc02322d6d23eca5049ab19bacd8ce539c88cd67ba33067
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=afa4ba0b826ee020638952f40f74a072c897fac1404f578c78100d73450c8877
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built o

In [2]:
!pip install flask-cors pyngrok

In [3]:
from pyngrok import ngrok
!ngrok authtoken 2rvCt0O3LY1ZQFX20uFGGcG4qZW_76is4e1n5YmKH7kpfUoXm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import threading
from pyngrok import ngrok
import os
from werkzeug.utils import secure_filename

import torch
import whisper
import gc
from tqdm import tqdm
import time
from datetime import datetime, timedelta

app = Flask(__name__)
CORS(app)

# Configuration for file uploads
UPLOAD_FOLDER = 'audio_uploads'
ALLOWED_EXTENSIONS = {'mp3', 'wav', 'ogg', 'm4a'}

# Create upload folder if it doesn't exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Sample data
books = [
    {"id": 1, "title": "The Great Gatsby", "author": "F. Scott Fitzgerald"},
    {"id": 2, "title": "1984", "author": "George Orwell"}
]




class OptimizedTranscriber:
    def __init__(self, model_name="large"):
        self.start_time = time.time()
        print(f"Initializing transcriber at {datetime.now().strftime('%H:%M:%S')}")

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Enable TensorFloat-32 for better performance on Ampere GPUs
        if torch.cuda.is_available():
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

        # Load model with optimizations
        self.model = whisper.load_model(model_name)
        self.model.to(self.device)

        if torch.cuda.is_available():
            # Convert model to half precision
            self.model = self.model.half()

        self.model_load_time = time.time() - self.start_time
        print(f"Model loading took: {timedelta(seconds=self.model_load_time)}")

    def transcribe(self, audio_path):
        start_time = time.time()
        print(f"\nStarting transcription of {audio_path} at {datetime.now().strftime('%H:%M:%S')}")

        try:
            # Enable cuda graphs for repeated operations
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            # Use modern autocast syntax
            with torch.amp.autocast('cuda' if torch.cuda.is_available() else 'cpu'), \
                 torch.no_grad():

                result = self.model.transcribe(
                    audio_path,
                    fp16=torch.cuda.is_available(),
                    verbose=True
                )

            # Force CUDA synchronization
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            process_time = time.time() - start_time
            print(f"Transcription completed in: {timedelta(seconds=process_time)}")

            return result, process_time

        finally:
            # Clean up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()

    def process_multiple(self, audio_files):
        """Process multiple audio files sequentially with progress bar"""
        results = []
        total_time = 0
        start_batch = time.time()

        for audio_file in tqdm(audio_files):
            result, process_time = self.transcribe(audio_file)
            results.append(result)
            total_time += process_time

        batch_time = time.time() - start_batch
        print(f"\nBatch Processing Summary:")
        print(f"Total files processed: {len(audio_files)}")
        print(f"Total processing time: {timedelta(seconds=batch_time)}")
        print(f"Average time per file: {timedelta(seconds=batch_time/len(audio_files))}")

        return results

def format_time(seconds):
    return str(timedelta(seconds=round(seconds)))

# Usage
# def main():
total_start = time.time()

# Initialize transcriber
transcriber = OptimizedTranscriber(model_name="large")

# Single file transcription
# result, process_time = transcriber.transcribe("/content/full.mp3")

# print("\n--- Full Transcription ---")
# print(result["text"])

# print("\n--- Transcription Details ---")
# print(f"Language Detected: {result['language']}")
# print(f"Approximate Duration: {len(result['segments'])} segments")
# print(f"Processing Time: {format_time(process_time)}")

# total_time = time.time() - total_start
# print(f"\nTotal execution time (including model loading): {format_time(total_time)}")

    # For processing multiple files:
    # audio_files = ["/content/1.mp3", "/content/2.mp3", ...]
    # results = transcriber.process_multiple(audio_files)

# if __name__ == "__main__":
#     main()




def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def home():
    return "Hello from Flask API"

@app.route('/api/books', methods=['GET'])
def get_books():
    return jsonify(books)

@app.route('/api/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book is None:
        return jsonify({"error": "Book not found"}), 404
    return jsonify(book)

@app.route('/api/books', methods=['POST'])
def create_book():
    if not request.is_json:
        return jsonify({"error": "Content-Type must be application/json"}), 400

    data = request.get_json()

    if not all(key in data for key in ['title', 'author']):
        return jsonify({"error": "Title and author are required"}), 400

    new_id = max(book['id'] for book in books) + 1
    new_book = {
        'id': new_id,
        'title': data['title'],
        'author': data['author']
    }

    books.append(new_book)
    return jsonify(new_book), 201

@app.route('/api/upload/audio', methods=['POST'])
def upload_audio():
    # Check if a file was included in the request
    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    file = request.files['audio']

    # Check if a file was selected
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    # Check if the file type is allowed
    if not allowed_file(file.filename):
        return jsonify({
            "error": f"File type not allowed. Allowed types: {', '.join(ALLOWED_EXTENSIONS)}"
        }), 400

    # Secure the filename and save the file
    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    result, process_time = transcriber.transcribe("/content/full.mp3")
    print('---------')
    print('---------')
    print(result)
    print('---------')
    print('---------')

    try:
        file.save(file_path)
        return jsonify({
            "message": "File uploaded successfully",
            "filename": filename,
            "path": f"/api/audio/{filename}"
        }), 201
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/audio/<filename>', methods=['GET'])
def get_audio(filename):
    try:
        return send_from_directory(app.config['UPLOAD_FOLDER'], filename)
    except FileNotFoundError:
        return jsonify({"error": "Audio file not found"}), 404

# Start ngrok when the app starts
def run_ngrok():
    public_url = ngrok.connect(5000)
    print(f" * Public URL: {public_url}")

if __name__ == '__main__':
    # Start ngrok in a new thread
    threading.Thread(target=run_ngrok).start()

    # Run the Flask application
    app.run(host='0.0.0.0', port=5000)

Initializing transcriber at 16:41:46
Using device: cuda


100%|██████████████████████████████████████| 2.88G/2.88G [00:27<00:00, 112MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Model loading took: 0:00:54.680136
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * Public URL: NgrokTunnel: "https://940c-35-221-203-99.ngrok-free.app" -> "http://localhost:5000"

Starting transcription of /content/full.mp3 at 16:44:57
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.640 --> 00:04.400]  Albert Einstein grew up in a middle class family.
[00:04.400 --> 00:11.920]  When he was born in his family was scared that something was wrong with him and he had
[00:11.920 --> 00:19.120]  very large in miss shape and head.
[00:19.120 --> 00:27.000]  But fortunately within the first few weeks the shape of his head became normal.
[00:27.000 --> 00:32.620]  But their worries didn't stop there when he was very young.
[00:32.620 --> 00:40.760]  His parents thought he might be intellectually disabled because he was very slow to learn
[00:40.760 --> 00:49.680]  to talk and did not speak until he was 4 years old.
[00:49.680 --> 00:56.980]  At that time he often formed foody sentences in his t

...

ERROR:__main__:Exception on /api/upload/audio [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(*

[00:00.640 --> 00:07.240]  Albert Einstein grew grew up in a middle class family when he was born born born was
[00:07.240 --> 00:16.600]  scared that something was wrong with him and he had very large in miss shape very large in miss
[00:16.600 --> 00:25.660]  shape and head head but fortunately within the first few weeks the shape of his head became
[00:25.660 --> 00:33.720]  normal but their worries didn't stop there when he was very young his parents through
[00:33.720 --> 00:36.980]  he might be intellectual
[00:36.980 --> 00:37.220]  intellectual
[00:38.100 --> 00:43.860]  well because he was very slow to learn to very slow to learn
[00:43.860 --> 00:51.120]  to talk and did not speak until he was four years old at that time he often
[00:51.120 --> 00:58.380]  formed full his n dances in his throat but did not speak his used practice the
[00:58.380 --> 01:05.520]  sentence in his head or whisper then softly under his breath until he got
[01:05.520 --> 01:06.980]  them right and t

INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 17:00:57] "POST /api/upload/audio HTTP/1.1" 201 -


[02:31.320 --> 02:33.800]  Albert Einstein.
Transcription completed in: 0:00:52.224302
---------
---------
{'text': " Albert Einstein grew grew up in a middle class family when he was born born born was scared that something was wrong with him and he had very large in miss shape very large in miss shape and head head but fortunately within the first few weeks the shape of his head became normal but their worries didn't stop there when he was very young his parents through he might be intellectual intellectual well because he was very slow to learn to very slow to learn to talk and did not speak until he was four years old at that time he often formed full his n dances in his throat but did not speak his used practice the sentence in his head or whisper then softly under his breath until he got them right and then very slow to learn to talk and did not speak until he was 4 years old at that time he was Then, say them aloud, many people believed Einstein would never succeed at anything. 

ERROR:__main__:Exception on /api/upload/audio [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(*